# Interactive Map — SpaceX Launches (Folium)

**Author:** Iliya Pezeshki  
**Course:** IBM Data Science Professional Certificate — Capstone  

**Objective:** Build an interactive Folium map showing SpaceX launch sites and missions using markers, circles, clustered markers, and optional route lines.

## 1) Setup & Load Data

In [2]:
!pip install folium

import pandas as pd, numpy as np
import folium
from folium.plugins import MarkerCluster

# Load cleaned dataset if available; otherwise use the IBM CSV (limited fields)
try:
    df = pd.read_csv('data/spacex_launches_clean.csv')
except FileNotFoundError:
    df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv')
    # Normalize a few expected column names
    df = df.rename(columns={
        'Launch_Site': 'LaunchSite',
        'PAYLOAD_MASS__KG_': 'PayloadMass',
        'Class': 'Class' if 'Class' in df.columns else 'Mission_Outcome'
    })

df.head(3)

,FlightNumber,MissionName,DateUTC,Rocket,LaunchSite,PayloadMass,Orbit,LandingType,LandingPad,Class
0,1,FalconSat,2006-03-24T22:30:00.000Z,Falcon 1,Kwajalein Atoll,20.0,LEO,NaN,NaN,0
1,2,DemoSat,2007-03-21T01:10:00.000Z,Falcon 1,Kwajalein Atoll,0.0,LEO,NaN,NaN,0
2,3,Trailblazer,2008-08-03T03:34:00.000Z,Falcon 1,Kwajalein Atoll,0.0,LEO,NaN,NaN,0


## 2) Coordinate Lookup for Launch Sites

In [3]:
site_coords = {
    'CCAFS SLC 40': (28.561857, -80.577366),
    'CCAFS LC-40': (28.561857, -80.577366),
    'KSC LC 39A':  (28.608389, -80.604333),
    'KSC LC-39A':  (28.608389, -80.604333),
    'VAFB SLC 4E': (34.632093, -120.610829),
    'VAFB SLC-4E': (34.632093, -120.610829)
}

df['lat'] = df['LaunchSite'].map(lambda s: site_coords.get(str(s), (np.nan, np.nan))[0])
df['lon'] = df['LaunchSite'].map(lambda s: site_coords.get(str(s), (np.nan, np.nan))[1])

print('Rows with coordinates:', df['lat'].notna().sum())

Rows with coordinates: 88


## 3) Create Base Map

In [4]:
start_location = (28.5, -80.6)
if df['lat'].notna().any():
    start_location = (df['lat'].mean(), df['lon'].mean())

m = folium.Map(location=start_location, zoom_start=5, tiles='OpenStreetMap', control_scale=True)
m

## 4) Add Site Circles (context radius)

In [5]:
for site, sub in df.dropna(subset=['lat','lon']).groupby('LaunchSite'):
    lat = sub['lat'].iloc[0]; lon = sub['lon'].iloc[0]
    folium.Circle(location=(lat,lon), radius=1500, popup=f"{site} (site area)", tooltip=site).add_to(m)

m

## 5) Add Mission Markers (MarkerCluster)

In [6]:
cluster = MarkerCluster(name='Launches').add_to(m)

def _payload(row):
    for key in ['PayloadMass','payload_mass','PAYLOAD_MASS__KG_']:
        if key in row and pd.notna(row[key]):
            return row[key]
    return None

def _orbit(row):
    for key in ['Orbit','orbit']:
        if key in row and pd.notna(row[key]):
            return row[key]
    return None

def _class(row):
    if 'Class' in row:
        return row['Class']
    if 'Mission_Outcome' in row:
        return 1 if str(row['Mission_Outcome']).startswith('Success') else 0
    return None

for _, r in df.dropna(subset=['lat','lon']).iterrows():
    label = (
        f"<b>Launch Site:</b> {r.get('LaunchSite', '')}<br>"
        f"<b>Orbit:</b> {_orbit(r)}<br>"
        f"<b>Payload (kg):</b> {_payload(r)}<br>"
        f"<b>Outcome/Class:</b> {_class(r)}"
    )
    folium.Marker(
        location=(r['lat'], r['lon']),
        popup=folium.Popup(label, max_width=300),
        tooltip=r.get('LaunchSite', 'Launch')
    ).add_to(cluster)

m

## 6) Optional Context Lines (example polyline)

In [7]:
if all(k in site_coords for k in ['CCAFS SLC 40','KSC LC 39A']):
    folium.PolyLine(
        locations=[site_coords['CCAFS SLC 40'], site_coords['KSC LC 39A']],
        weight=2, opacity=0.6, tooltip='Cape Canaveral ↔ KSC LC-39A'
    ).add_to(m)
m

## 7) Save Map

In [ ]:
Path('maps').mkdir(exist_ok=True, parents=True)
out_path = 'maps/spacex_launch_map.html'
m.save(out_path)
print('Saved interactive map →', out_path)

---
### Notes
- Map objects used: **Circle** (site context), **Marker** (mission), **MarkerCluster** (declutter overlapping markers), **PolyLine** (illustrative path).
- Open the saved HTML (`maps/spacex_launch_map.html`) in a browser and take screenshots for your slides.